In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from gensim.models import KeyedVectors

In [2]:
from src.data_processing.process_labels import *
from src.data_processing.process_reviews import *

## Load in saved LSTM torch model

In [5]:
from src.models.model_zoo import *
from src.models.model_train import *

# instatiate model
input_size = 300
hidden_size = 50
num_layers = 1
dropout_prob = 0.3
output_size = 4
lstm_model = LSTMmodel(input_size=input_size, hidden_size=hidden_size, 
                       output_size=output_size, num_layers=num_layers, dropout_prob=dropout_prob)

c:\Users\mattc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
# load saved parameters
lstm_model.load_state_dict(torch.load('jobs/out_h50_l1_dp03.0.pt'))
lstm_model.eval()

LSTMmodel(
  (LSTM): LSTM(300, 50, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Linear(in_features=100, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

## Sanity check: try made up review

In [9]:
# load word2vec model
model = KeyedVectors.load('word2vec/word2vec-google-news-300.model') # Load word2vec model

In [12]:
madeup = ['food food food food food']
lstm_model(process_reviews_w2v(madeup, model))

Fetching review embeddings: 100%|██████████| 1/1 [00:00<00:00, 250.06it/s]


tensor([[0.2865, 0.1258, 0.4603, 0.1273]], grad_fn=<SoftmaxBackward0>)